In [1]:
import sys
sys.path.append("..")

from libs.InteractivePitchUI import *
from libs.data_manipulation import *
from libs.dim_reduction import *
from libs.feature_generation import *
from libs.clustering import *
from libs.convex_hull import *
from libs.alpha_shape import *
from libs.weight_generator import *
from libs.similar_movement import *
from libs.Video_Player import *

In [2]:
matches = compile_team_tracking_data_with_labels("../data/H_EURO2024GERMANY", "Denmark", "../data/Labelled_ground_truths.csv")




In [ ]:
df = extract_one_match(matches,4)

In [ ]:
df.head()
df.columns

Index(['match_id', 'Time [s]', 'Time [s]_team', 'half_team', 'minute_team',
       'Period_team', 'home_6_x', 'home_6_y', 'home_18_x', 'home_18_y',
       ...
       'away_25_y', 'home_26_x', 'home_26_y', 'away_3_x', 'away_3_y',
       'away_15_x', 'away_15_y', 'Label', 'Time[s]', 'Competition'],
      dtype='object', length=101)

In [ ]:
df = df[df["match_name"].isin(["Denmark_England","Slovenia_Denmark"])]




In [ ]:
func_control = lambda x:1
func_inverse = lambda x: 20/x
func_linear = lambda x: 200-x
func_exp = lambda x: np.exp(-x/20)

In [ ]:
df.Label.unique()

array(['Missing', 'breakthrough-wb', 'run-into-space',
       'breakthrough-even', 'breakthrough-space'], dtype=object)

In [ ]:
import datetime


def process_situations(df):
    # Dictionary to store recommendations
    recommendations_dict = {}

    # Step 2: Extract relevant rows (all rows with a label not equal to "Missing")
    labeled_rows = df[df['Label'] != "Missing"]
    relevant_indices = labeled_rows.index.tolist()
    print(relevant_indices)
    # Step 2: Process each relevant index
    # Step 2: Process each relevant index
    for relevant_index in relevant_indices:
        # Step 3: Apply Wasserstein distance for various functions upfront
        finished_indices_control = most_similar_with_wasserstein_closed_interval(relevant_index, df, func_control)
        finished_indices_linear = most_similar_with_wasserstein_closed_interval(relevant_index, df, func_linear)
        finished_indices_inverse = most_similar_with_wasserstein_closed_interval(relevant_index, df, func_inverse)
        finished_indices_exp = most_similar_with_wasserstein_closed_interval(relevant_index, df, func_exp)

        # Convert to numpy arrays
        finished_indices_control = np.array(finished_indices_control)
        finished_indices_exp = np.array(finished_indices_exp)
        finished_indices_linear = np.array(finished_indices_linear)
        finished_indices_inverse = np.array(finished_indices_inverse)

        # Step 4: Filter indices based on the time window (±480 seconds)
        time_window_filter = lambda indices: indices[
            (indices > relevant_index + 480) | 
            (indices < relevant_index - 480)
        ]

        finished_indices_control = time_window_filter(finished_indices_control)[:100]
        finished_indices_exp = time_window_filter(finished_indices_exp)[:100]
        finished_indices_linear = time_window_filter(finished_indices_linear)[:100]
        finished_indices_inverse = time_window_filter(finished_indices_inverse)[:100]

        # Combine recommendations into a dictionary for separate processing
        wasserstein_recommendations = {
            "Control": finished_indices_control,
            "Exp": finished_indices_exp,
            "Linear": finished_indices_linear,
            "Inverse": finished_indices_inverse
        }
        # Create the folder based on the label
        # Step 6: Prepare the result DataFrame for this function set
        label = df.loc[relevant_index, 'Label']  # Get the label for the situation
        match_name = df.loc[relevant_index, 'match_name']
        time_of_situation = str(datetime.timedelta(seconds=df.loc[relevant_index, 'Time [s]']))

        folder_name = f"{label}_{match_name}_{time_of_situation}"
        generate_folder(folder_name)

        # Save the time and match_name to a text file
        with open(f"{folder_name}/situation_info.txt", "w") as f:
                f.write(f"Match Name: {match_name}\n")
                f.write(f"Time of Situation: {time_of_situation}\n")

        # Step 5: Process each set of indices separately
        for func_name, indices_set in wasserstein_recommendations.items():
            if len(indices_set) == 0:
                continue  # Skip if no indices in this set

            # Step 5.1: Compute similar movements for the current set
            similar_movement = find_similar_movement(df, relevant_index, indices_set,length_of_sequence=120)
            similar_movement_ranked = sorted(similar_movement, key=lambda x: x[0])

            # Step 5.2: Select indices, ensuring no two are within 100 places of each other
            selected_indices = []
            for _, index in similar_movement_ranked:
                if all(abs(index - selected) > 500 for selected in selected_indices):
                    selected_indices.append(index)
                if len(selected_indices) == 10:  # Stop once we have 10 indices
                    break

            new_df = df.loc[selected_indices[:10]]

            
            


            # Generate the result DataFrame
            result_df = new_df[["match_name", "Time [s]", "half_team"]]
            result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))

            # Save the ground truth CSV for this function set
            result_df.to_csv(f"{folder_name}/{folder_name}_{func_name}_ground_truth.csv")

            # Shuffle and save the ordering truth CSV for this function set
            result_df.sample(frac=1).to_csv(f"{folder_name}/{folder_name}_{func_name}_for_ordering_truth.csv")

            # Store the recommendations in the dictionary
            recommendations_dict[relevant_index] = recommendations_dict.get(relevant_index, {})
            recommendations_dict[relevant_index][func_name] = selected_indices[:10]

    # Return the dictionary with recommendations
    return recommendations_dict

    

reccomendations_dict = process_situations(df)

[1675, 8725, 12300, 39000, 48500, 85241, 93791, 129716, 143547, 154347, 181797, 189572, 199447, 208722, 214138]
21284
21284
21284
21284
Folder 'breakthrough-wb_Slovenia_Denmark_0:01:07' does not exist.


/tmp/ipykernel_4115/259074268.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))
/tmp/ipykernel_4115/259074268.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))
/tmp/ipykernel_4115/259074268.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

21284
21284
21284
21284
Folder 'run-into-space_Slovenia_Denmark_0:05:49' does not exist.


/tmp/ipykernel_4115/259074268.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))
/tmp/ipykernel_4115/259074268.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))
/tmp/ipykernel_4115/259074268.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

21284
21284
21284
21284
Folder 'breakthrough-wb_Slovenia_Denmark_0:08:12' does not exist.


...

/tmp/ipykernel_4115/259074268.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))
/tmp/ipykernel_4115/259074268.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))
/tmp/ipykernel_4115/259074268.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

21284
21284
21284
21284
Folder 'run-into-space_Denmark_England_0:43:37' does not exist.


/tmp/ipykernel_4115/259074268.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))
/tmp/ipykernel_4115/259074268.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))
/tmp/ipykernel_4115/259074268.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

21284
21284
21284
21284
Folder 'breakthrough-even_Denmark_England_0:46:06' does not exist.


/tmp/ipykernel_4115/259074268.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))
/tmp/ipykernel_4115/259074268.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["Time Min"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))
/tmp/ipykernel_4115/259074268.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [ ]:
reccomendations_dict

{1675: {'Control': [62112,
   128040,
   2880,
   165408,
   149796,
   52332,
   39084,
   189300,
   6228,
   259068],
  'Exp': [125112,
   273372,
   62088,
   172572,
   258972,
   1188,
   204324,
   128040,
   243720,
   2880],
  'Linear': [62112,
   128040,
   2880,
   165408,
   149796,
   52332,
   39084,
   189300,
   6228,
   259068],
  'Inverse': [125124,
   273408,
   62088,
   258984,
   1188,
   128040,
   243720,
   172584,
   2880,
   68364]},
 8725: {'Control': [143808,
   139620,
   154392,
   189552,
   187872,
   73404,
   201312,
   39096,
   173892,
   63084],
  'Exp': [97524,
   17628,
   278160,
   221904,
   15684,
   193632,
   44820,
   259260,
   143808,
   106380],
  'Linear': [143808,
   139620,
   154392,
   189552,
   187872,
   73404,
   201312,
   39096,
   173892,
   63084],
  'Inverse': [97524,
   44820,
   259260,
   143808,
   221916,
   73392,
   139620,
   187872,
   154380,
   39816]},
 12300: {'Control': [143844,
   154416,
   129816,
   45792

In [1]:
root ="csvs_for_ranking"
folders = os.listdir(root)

import re

def extract_seconds_from_folder(folder_name):
    # Use regex to find the time pattern (hh:mm:ss)
    match = re.search(r'(\d+):(\d+):(\d+)', folder_name)
    if match:
        hours, minutes, seconds = map(int, match.groups())
        # Convert to total seconds
        return hours * 3600 + minutes * 60 + seconds
    else:
        # Return None if no time is found
        return None



for folder in folders:
    csvs = os.listdir(root + "/"+folder)
    for csv in csvs:
        if ("ground_truth" in csv):
            print(folder)


            seconds = extract_seconds_from_folder(folder)

            if (seconds > 2700 and "England" in csv):
                myplayer = VideoPlayer("3-2024-2036178-Denmark_England.mp4",folder,8+977+seconds,0,[])
            if (seconds < 2700 and "England" in csv):
                myplayer = VideoPlayer("3-2024-2036178-Denmark_England.mp4",folder,8+seconds,0,[])
            if (seconds > 2700 and "Slovenia" in csv):
                myplayer = VideoPlayer("3-2024-2036165-slovenia-denmark.mp4",folder,8+954+seconds,0,[])
            if (seconds < 2700 and "Slovenia" in csv):
                myplayer = VideoPlayer("3-2024-2036165-slovenia-denmark.mp4",folder,8+seconds,0,[])
            
            
            
            df = pd.read_csv(root +"/"+folder+"/"+csv)
            #Match_name, time, half
            indices = df[["Time [s]","half_team",'Unnamed: 0', 'match_name']]
            ratings = []
            matches = df.match_name.unique()
            for match_name in matches:
                if "England" in match_name: 
                    myplayer = VideoPlayer("3-2024-2036178-Denmark_England.mp4",folder,8,8+977,indices[indices['match_name'] == match_name].to_numpy())
                    [ratings.append(row) for row in myplayer.ratings]
                if "Slovenia" in match_name:
                    myplayer = VideoPlayer("3-2024-2036165-slovenia-denmark.mp4",folder,8,8+954,indices[indices['match_name'] == match_name].to_numpy())
                    [ratings.append(row) for row in myplayer.ratings]
            if (len(ratings) > 0):
                ratings_df  = pd.DataFrame(ratings, columns=['Unnamed: 0',"rating"])
                rated_situations = pd.merge(df, ratings_df, how ="inner", on=['Unnamed: 0'])
                rated_situations =rated_situations.drop_duplicates('Unnamed: 0')
                rated_situations.to_csv(root +"/"+folder+"/"+"ranked_"+csv)




NameError: name 'os' is not defined

In [ ]:
ratings_df  = pd.DataFrame(ratings, columns=['Unnamed: 0',"rating"])

NameError: name 'ratings' is not defined

In [ ]:
rated_situations = pd.merge(df, ratings_df, how ="inner", on=['Unnamed: 0'])

In [ ]:
rated_situations =rated_situations.drop_duplicates('Unnamed: 0')

In [ ]:
print(rated_situations)

    Unnamed: 0       match_name  Time [s] half_team        Time Min  rating
0       179537  Denmark_England   1449.60        1H  0:24:09.600000       1
1       174689  Denmark_England   1255.68        1H  0:20:55.680000       0
3       199505  Denmark_England   2248.32        1H  0:37:28.320000       1
5       149921  Denmark_England    264.96        1H  0:04:24.960000       1
6       193697  Denmark_England   2016.00        1H         0:33:36       0
7       208001  Denmark_England   2588.16        1H  0:43:08.160000       1
8       238961  Denmark_England   3758.92        2H  1:02:38.920000       0
9       253697  Denmark_England   4348.36        2H  1:12:28.360000       0
10      176321  Denmark_England   1320.96        1H  0:22:00.960000       1
11      185441  Denmark_England   1685.76        1H  0:28:05.760000       1
